### Loading documents

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "/home/cristian/projects/rag_pae/corpus.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

156


#### Calling the LLM

In [17]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.2")

In [19]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=splits, embedding= embeddings, persist_directory="chroma_db")
retriever = vectorstore.as_retriever()

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": """

    

"""})

results['answer']

'No tengo información suficiente para responder a esta pregunta. No se proporcionó suficiente contexto o detalles para determinar qué pregunta específica se está haciendo. ¿Puedo ayudarte con algo más?'